In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
import numpy as np
import noise

from matplotlib import pyplot as plt
from matplotlib import colors as col
from mpl_toolkits.mplot3d import Axes3D

from ipywidgets import IntProgress, HTML, VBox
from IPython.display import display
import ipywidgets as wdg

import warnings
warnings.filterwarnings("ignore")

from plotly import graph_objs as plgo
from plotly import offline as ploff
ploff.init_notebook_mode(connected=False)

In [ ]:
def plot(x, y, land, water, title):
#     mask = ~water.astype(bool)
    mask = water < 0.00001
    layer = land + water
    layer[mask] = None
    ploff.iplot(
        plgo.Figure({
            'data': [
                plgo.Surface({
                    'name': 'land',
                    'x': x,
                    'y': y,
                    'z': land,
                    'colorscale': [(0.0, '#F9EBEA'), (1.0, '#641E16')],
                }),
                plgo.Surface({
                    'name': 'water',
                    'x': x,
                    'y': y,
                    'z': layer,
                    'colorscale': [(0.0, '#154360'), (1.0, '#85C1E9')],
                    'showscale': False,
                })
            ],
            'layout': plgo.Layout({
                'title': title,
                'width': 800,
                'height': 800,
                'scene': plgo.Scene({
                    'zaxis': plgo.ZAxis({
                        'range': [-1, 1.5]
                    })
                })
            }),
        })
    )

In [ ]:
def upper_plot(x, y, land, water, Timestep):
    mask = water[Timestep] < 0.00001
    layer = land + water[Timestep]
    layer[mask] = None
    land_cm = col.LinearSegmentedColormap.from_list(
        'land',
        [(110/255, 44/255, 0), (246/255, 221/255, 204/255)][::-1]
    )
    water_cm = col.LinearSegmentedColormap.from_list(
        'water',
        [(27/255, 79/255, 144/255), (214/255, 234/255, 248/255)]
    )
    fig = plt.figure(figsize=(16,8))
    ax1 = fig.add_subplot(122)
    ax1.imshow(land, cmap=land_cm, vmin=np.min(land), vmax=np.max(land))
    ax1.imshow(layer, cmap=water_cm, vmin=np.min(land), vmax=np.max(land))
    ax2 = fig.add_subplot(121, projection='3d')
    ax2.plot_surface(x, y, land, cmap=land_cm)
    ax2.plot_surface(x, y, layer, cmap=water_cm)
    ax2.set_zlim(-1, 1.5)
    plt.show()

In [ ]:
np.random.seed(42)

# Flooding Model
---

by Francisco Casas & Vicente Lizana

## Terrain Generation
---

In [ ]:
base = np.linspace(0, 5, 100)
x, y = np.meshgrid(base, base)
# vectoriation of perlin noise
# final_func = np.vectorize(lambda x, y: noise.pnoise2(x, y, octaves=4))
# land = final_func(x, y)
land = np.zeros(x.shape)

In [ ]:
plot(x, y, land, np.zeros(land.shape), 'Terrain')

## First Model
### Simple automata
---

The water transference between two adjacent cells $i$ and $j$ is: $$ Q_{ij} = \begin{cases} -\alpha \min(w_i, (t_i + w_i) - (t_j + w_j)), \quad & t_i + w_i \geqslant t_j + w_j\\ +\alpha \min(w_j, (t_j + w_j) - (t_i + w_i)), \quad & t_i + w_i < t_j + w_j \end{cases} $$

where $\alpha$ is a constant in $[0,0.25]$.

In [ ]:
def simple_automata_step(terra,water,alpha):
    assert(terra.shape==water.shape)
    levels = terra+water
    # Get the level transfer:
    transfer_x = levels[:,:-1]-levels[:,1:]
    transfer_x = np.minimum(water[:,:-1],transfer_x)
    transfer_x = np.maximum(-water[:,1:],transfer_x)
    #
    transfer_y = levels[:-1,:]-levels[1:,:]
    transfer_y = np.minimum(water[:-1,:],transfer_y)
    transfer_y = np.maximum(-water[1:,:],transfer_y)
    # Update water levels:
    nwater = water.copy()
    nwater[:,:-1] -= transfer_x*alpha
    nwater[:,1:]  += transfer_x*alpha
    nwater[:-1,:] -= transfer_y*alpha
    nwater[1:,:]  += transfer_y*alpha
    return nwater

In [ ]:
water = np.zeros(land.shape)
water[45:56,45:56] = 0.5 - land[45:56,45:56]

In [ ]:
size = 1000
alpha = 0.2

w = np.zeros((size, *land.shape))
w[0,:,:] = simple_automata_step(land, water, alpha)
for i in log_progress(range(size-1), every=1, name='Timesteps'):
    w[i+1,:,:] = simple_automata_step(land, w[i,:,:], alpha)

In [ ]:
play = wdg.Play(
#     interval=10,
    value=0,
    min=0,
    max=size-1,
    step=1,
    description="Press play",
    disabled=False
)
slider = wdg.IntSlider(
    min=0,
    max=size
)
wdg.jslink((play, 'value'), (slider, 'value'))
flooding = wdg.interactive(upper_plot, Timestep=slider, water=wdg.fixed(w), land=wdg.fixed(land), x=wdg.fixed(x), y=wdg.fixed(y))
display(wdg.VBox([play, flooding]))

<img src="moni.png">

In [ ]:
def weighted_automata_step(terra, water, transf, k=0.00001):
    assert(terra.shape==water.shape)
    levels = terra+water
    levelsb = np.zeros((levels.shape[0]+2,levels.shape[1]+2))
    levelsb[1:-1,1:-1] = levels
    levelsb[0,:] = levelsb[1,:]
    levelsb[-1,:] = levelsb[-2,:]
    levelsb[:,0] = levelsb[:,-1]
    levelsb[:,-1] = levelsb[:,-2]
    # Get water level height difference on each direction
    delta_1 = levels-levelsb[2:,1:-1]
    delta_1 = np.min([delta_1,water],axis=0)
    delta_2 = levels-levelsb[1:-1,:-2]
    delta_2 = np.min([delta_2,water],axis=0)
    delta_3 = levels-levelsb[:-2,1:-1]
    delta_3 = np.min([delta_3,water],axis=0)
    delta_4 = levels-levelsb[1:-1,2:]
    delta_4 = np.min([delta_4,water],axis=0)
    # Filter small and negative discharges
    delta_1[delta_1 < k] = 0
    delta_2[delta_2 < k] = 0
    delta_3[delta_3 < k] = 0
    delta_4[delta_4 < k] = 0
    # Get the smallest height difference
    diffs = np.array([delta_1[:,:], delta_2[:,:], delta_3[:,:], delta_4[:,:]])
    diffs[diffs<k] = float('inf')
    mindiff = np.min(diffs,axis=0)
    # Get the transfer weights
    diffs = np.sum([delta_1, delta_2, delta_3, delta_4, mindiff], axis=0)
    delta_1 /= diffs
    delta_2 /= diffs
    delta_3 /= diffs
    delta_4 /= diffs
    # Transfer flow
    nwater = water.copy()
    nwater[1:,:] += delta_1[:-1,:]
    nwater[:,1:] += delta_2[:,:-1]
    nwater[:-1,:] += delta_3[1:,:]
    nwater[:,:-1] += delta_4[:,1:]
    # Return new water level
    return nwater

In [ ]:
water = np.zeros(land.shape)
water[45:56,45:56] = 0.5 - land[45:56,45:56]

In [ ]:
size = 1000
alpha = 0.2

w = np.zeros((size, *land.shape))
w[0,:,:] = weighted_automata_step(land, water, alpha)
for i in log_progress(range(size-1), every=1, name='Timesteps'):
    w[i+1,:,:] = simple_automata_step(land, w[i,:,:], alpha)

In [ ]:
play = wdg.Play(
#     interval=10,
    value=0,
    min=0,
    max=size-1,
    step=1,
    description="Press play",
    disabled=False
)
slider = wdg.IntSlider(
    min=0,
    max=size
)
wdg.jslink((play, 'value'), (slider, 'value'))
flooding = wdg.interactive(upper_plot, Timestep=slider, water=wdg.fixed(w), land=wdg.fixed(land), x=wdg.fixed(x), y=wdg.fixed(y))
display(wdg.VBox([play, flooding]))